In [1]:
!pip install tensorflow matplotlib split-folders gradio


Let's test the model with a sample image from the test set.

In [2]:
!pip install -q --upgrade pip
!pip install -q tensorflow matplotlib split-folders gradio==4.8.8
print("✅ All packages installed")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 39.8 MB/s eta 0:00:00
ERROR: Ignored the following yanked versions: 2.8.0, 3.0.7, 3.1.2, 3.13.1, 3.45.0b9, 3.45.0b10, 3.45.0b11, 3.45.0b12, 3.45.0b13, 4.0.0b15, 4.7.0
ERROR: Could not find a version that satisfies the requirement gradio==4.8.8 (from versions: 0.1.0, 0.1.1, 0.1.2, 0.1.3, 0.1.4, 0.1.5, 0.1.6, 0.1.7, 0.1.8, 0.1.9, 0.2.0, 0.2.1, 0.3.0, 0.3.1, 0.3.2, 0.3.3, 0.3.4, 0.3.5, 0.4.0, 0.4.1, 0.4.2, 0.4.4, 0.5.0, 0.7.0, 0.7.1, 0.7.2, 0.7.3, 0.7.4, 0.7.5, 0.7.6, 0.7.7, 0.7.8, 0.8.0, 0.8.1, 0.9.0, 0.9.1, 0.9.2, 0.9.3, 0.9.4, 0.9.5, 0.9.6, 0.9.7, 0.9.8, 0.9.9.2, 0.9.9.3, 0.9.9.5, 0.9.9.6, 0.9.9.7, 0.9.9.8, 0.9.9.9, 0.9.9.9.2, 1.0.0a1, 1.0.0a3, 1.0.0a4, 1.0.0, 1.0.1, 1.0.2, 1.0.3, 1.0.4, 1.0.5, 1.0.6, 1.0.7, 1.1.0, 1.1.1, 1.1.2, 1.1.3, 1.1.4, 1.1.5, 1.1.6, 1.1.8, 1.1.8.1, 1.1.9, 1.2.2, 1.2.3, 1.3.0, 1.3.1, 1.3.2, 1.4.0, 1.4.2, 1.4.3, 1.4.4, 1.5.0, 1.5.1, 1.5.3, 1.5.4, 1.6.0, 1.6.1, 1.6.2, 1.6.3, 1.6.4, 1.7.0, 1.7.1, 1.7.2, 1.7.3, 1

In [3]:
import os, glob, tensorflow as tf

dataset_url = "https://github.com/Bangkit-JKT2-D/fruits-fresh-rotten-classification/archive/refs/heads/master.zip"
zip_path = tf.keras.utils.get_file("fruits_dataset.zip", origin=dataset_url, extract=True)

base_parent = os.path.dirname(zip_path)
base_dir = None
for root, dirs, files in os.walk(base_parent):
    for d in dirs:
        if "fruits-fresh-rotten-classification" in d:
            base_dir = os.path.join(root, d)
            break
    if base_dir:
        break

if not base_dir:
    raise FileNotFoundError("❌ Dataset folder not found")

print("Dataset found at:", base_dir)


1927667712/Unknown 77s 0us/stepDataset found at: /root/.keras/datasets/fruits_dataset_extracted/fruits-fresh-rotten-classification-master


In [4]:
import shutil

merged_dir = "binary_fruit_dataset"
fresh_dir = os.path.join(merged_dir, "fresh")
spoiled_dir = os.path.join(merged_dir, "spoiled")
os.makedirs(fresh_dir, exist_ok=True)
os.makedirs(spoiled_dir, exist_ok=True)

fresh_count = spoiled_count = 0

for root, dirs, files in os.walk(base_dir):
    folder = os.path.basename(root).lower()
    imgs = glob.glob(os.path.join(root, "*.jpg")) + glob.glob(os.path.join(root, "*.png"))
    for img in imgs:
        if "fresh" in folder:
            shutil.copy(img, os.path.join(fresh_dir, os.path.basename(img)))
            fresh_count += 1
        elif "rotten" in folder or "spoiled" in folder:
            shutil.copy(img, os.path.join(spoiled_dir, os.path.basename(img)))
            spoiled_count += 1

print(f"✅ Fresh: {fresh_count} | Spoiled: {spoiled_count}")


✅ Fresh: 5904 | Spoiled: 7695


In [5]:
import splitfolders
splitfolders.ratio(merged_dir, output="binary_split", seed=42, ratio=(.8, .1, .1))
print("✅ Split done (binary_split/train, val, test)")


Copying files: 13599 files [00:16, 822.53 files/s] 

✅ Split done (binary_split/train, val, test)


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (150,150)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True
)
val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    "binary_split/train", target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary')
val_gen = val_datagen.flow_from_directory(
    "binary_split/val", target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary')

print("Train:", train_gen.samples, " Val:", val_gen.samples)


Found 10879 images belonging to 2 classes.
Found 1359 images belonging to 2 classes.
Train: 10879  Val: 1359


In [7]:
from tensorflow.keras import layers, models

base = tf.keras.applications.MobileNetV2(
    input_shape=(150,150,3), include_top=False, weights='imagenet')
base.trainable = False

model = models.Sequential([
    base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


/tmp/ipython-input-3191289971.py:3: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base = tf.keras.applications.MobileNetV2(


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 5, 5, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [8]:
EPOCHS = 5
history = model.fit(train_gen, validation_data=val_gen, epochs=EPOCHS)
print("✅ Training finished")


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
340/340 ━━━━━━━━━━━━━━━━━━━━ 155s 400ms/step - accuracy: 0.9414 - loss: 0.1462 - val_accuracy: 0.9448 - val_loss: 0.1505
Epoch 2/5
340/340 ━━━━━━━━━━━━━━━━━━━━ 108s 319ms/step - accuracy: 0.9742 - loss: 0.0720 - val_accuracy: 0.9890 - val_loss: 0.0380
Epoch 3/5
340/340 ━━━━━━━━━━━━━━━━━━━━ 109s 321ms/step - accuracy: 0.9818 - loss: 0.0494 - val_accuracy: 0.9912 - val_loss: 0.0297
Epoch 4/5
340/340 ━━━━━━━━━━━━━━━━━━━━ 108s 317ms/step - accuracy: 0.9830 - loss: 0.0426 - val_accuracy: 0.9845 - val_loss: 0.0432
Epoch 5/5
340/340 ━━━━━━━━━━━━━━━━━━━━ 107s 316ms/step - accuracy: 0.9857 - loss: 0.0370 - val_accuracy: 0.9610 - val_loss: 0.1065
✅ Training finished


In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    "binary_split/test", target_size=IMG_SIZE, batch_size=1, class_mode='binary')

loss, acc = model.evaluate(test_gen, steps=min(50, test_gen.samples))
print(f"Loss={loss:.4f}, Accuracy={acc:.4f}")

model.save("fruit_fresh_spoiled_model.h5")
print("✅ Model saved")


Found 1361 images belonging to 2 classes.
50/50 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - accuracy: 1.0000 - loss: 0.0126


Loss=0.0149, Accuracy=1.0000
✅ Model saved


In [10]:
import gradio as gr
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model

# If you already trained earlier, this reloads saved model
model = load_model("fruit_fresh_spoiled_model.h5")

def classify_fruit(img):
    if isinstance(img, np.ndarray):
        img = Image.fromarray(img)
    img = img.convert("RGB").resize((150,150))
    x = np.expand_dims(np.array(img)/255.0, axis=0)
    pred = float(model.predict(x)[0][0])
    label = "🍏 Fresh" if pred < 0.5 else "🍂 Spoiled"
    conf = pred if pred >= 0.5 else 1 - pred
    return {label: float(conf)}

demo = gr.Interface(
    fn=classify_fruit,
    inputs=gr.Image(sources=["webcam", "upload"], label="📸 Use Laptop Camera"),
    outputs=gr.Label(),
    title="🍎 Fresh vs Spoiled Fruit Detector",
    description="Open your laptop camera and scan a fruit."
)

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c345d9eb10e991d2de.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Task
Install Streamlit and `pyngrok` for building and deploying the web application.

## Install Streamlit and NGROK

### Subtask:
Install the `streamlit` and `pyngrok` libraries required for building and deploying the web application.


**Reasoning**:
To achieve the subtask, I will install the `streamlit` and `pyngrok` libraries using pip.



In [11]:
pip install streamlit pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 42.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 102.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [streamlit]


## Create Streamlit Application

### Subtask:
Write the Python code for the Streamlit application. This will include loading the `fruit_fresh_spoiled_model.h5` model, defining the prediction function to classify fruits and provide confidence scores, and setting up the user interface with an image input (upload and/or webcam) and output for the classification result and confidence.


**Reasoning**:
I will create a new Python file named `streamlit_app.py` containing the Streamlit application code. This file will import required libraries, load the pre-trained model, define the prediction function for fruit classification, and set up the Streamlit UI for image upload and displaying predictions.



In [12]:
%%writefile streamlit_app.py
import streamlit as st
import numpy as np
import tensorflow as tf
from PIL import Image

# Load the pre-trained model
# Ensure the model file is accessible (e.g., in the same directory or specified path)
MODEL_PATH = 'fruit_fresh_spoiled_model.h5'

@st.cache_resource
def load_my_model():
    model = tf.keras.models.load_model(MODEL_PATH)
    return model

model = load_my_model()

IMG_SIZE = (150, 150)

def classify_fruit_streamlit(img):
    """Predicts whether a fruit image is fresh or spoiled."""
    # Ensure image is in PIL format for consistent processing
    if isinstance(img, np.ndarray):
        img = Image.fromarray(img)

    # Resize and convert to RGB
    img = img.convert("RGB").resize(IMG_SIZE)

    # Normalize and expand dimensions
    x = np.expand_dims(np.array(img) / 255.0, axis=0)

    # Make prediction
    prediction = model.predict(x)[0][0]

    # Determine label and confidence
    if prediction < 0.5:
        label = "🍏 Fresh"
        confidence = 1 - prediction
    else:
        label = "🍂 Spoiled"
        confidence = prediction

    return label, confidence

# Streamlit App Layout
st.title("🍎 Fresh vs Spoiled Fruit Detector")
st.write("Upload an image of a fruit to classify it as fresh or spoiled.")

uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    # Display the uploaded image
    image = Image.open(uploaded_file)
    st.image(image, caption='Uploaded Image', use_column_width=True)
    st.write("")

    # Make prediction
    label, confidence = classify_fruit_streamlit(image)

    # Display results
    st.write("### Prediction:")
    if "Fresh" in label:
        st.success(f"The fruit is {label} with {confidence*100:.2f}% confidence.")
    else:
        st.error(f"The fruit is {label} with {confidence*100:.2f}% confidence.")


Writing streamlit_app.py


## Configure NGROK Tunnel

### Subtask:
Initialize and authenticate NGROK using the provided NGROK authentication token to create a public URL for the Streamlit application.


### Get your NGROK Authentication Token

1.  **Sign up/Log in to ngrok**: If you don't have an ngrok account, sign up at [ngrok.com](https://ngrok.com). If you do, log in.
2.  **Access your Dashboard**: Once logged in, navigate to your ngrok dashboard.
3.  **Find your Auth Token**: Your authentication token will be prominently displayed on the dashboard, usually under 'Your Authtoken'. It typically looks like `2k2sV4F2o3eW2p5x0y7fX2dZ0q3aG7gJ1w2Q6z4r5D6F9j9C0N`. Copy this token.

**Reasoning**:
Now that the `streamlit` and `pyngrok` libraries are installed, the next step is to import `pyngrok` and set the authentication token using `ngrok.set_auth_token()`. This will allow `pyngrok` to establish a secure tunnel.



In [13]:
from pyngrok import ngrok

# IMPORTANT: Replace 'YOUR_NGROK_AUTH_TOKEN' with your actual ngrok authentication token
NGROK_AUTH_TOKEN = '34kbOT6L1mXAIZVVQc5MxsSN22g_4bUZKAtvNLpKRcbyY3ncY'
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

print("✅ ngrok authentication token set")

✅ ngrok authentication token set


**Reasoning**:
Now that the ngrok authentication token is set, the next step is to start the Streamlit application and then establish an ngrok tunnel to expose it to the internet. This will make the Streamlit app accessible via a public URL.



In [14]:
%%writefile streamlit_app.py
import streamlit as st
import numpy as np
import tensorflow as tf
from PIL import Image

# Load the pre-trained model
# Ensure the model file is accessible (e.g., in the same directory or specified path)
MODEL_PATH = 'fruit_fresh_spoiled_model.h5'

@st.cache_resource
def load_my_model():
    model = tf.keras.models.load_model(MODEL_PATH)
    return model

model = load_my_model()

IMG_SIZE = (150, 150)

def classify_fruit_streamlit(img):
    """Predicts whether a fruit image is fresh or spoiled."""
    # Ensure image is in PIL format for consistent processing
    if isinstance(img, np.ndarray):
        img = Image.fromarray(img)

    # Resize and convert to RGB
    img = img.convert("RGB").resize(IMG_SIZE)

    # Normalize and expand dimensions
    x = np.expand_dims(np.array(img) / 255.0, axis=0);

    # Make prediction
    prediction = model.predict(x)[0][0]

    # Determine label and confidence
    if prediction < 0.5:
        label = "🍏 Fresh"
        confidence = 1 - prediction
    else:
        label = "🍂 Spoiled"
        confidence = prediction

    return label, confidence

# Streamlit App Layout
st.title("🍎 Fresh vs Spoiled Fruit Detector")
st.write("Upload an image of a fruit or use your camera to classify it as fresh or spoiled.")

uploaded_file = st.file_uploader("Choose an image from your files...", type=["jpg", "jpeg", "png"])
camera_image = st.camera_input("Take a picture with your camera")

image_to_process = None
if uploaded_file is not None:
    image_to_process = Image.open(uploaded_file)
elif camera_image is not None:
    image_to_process = Image.open(camera_image)

if image_to_process is not None:
    # Display the selected image
    st.image(image_to_process, caption='Selected Image', use_container_width=True)
    st.write("")

    # Make prediction
    label, confidence = classify_fruit_streamlit(image_to_process)

    # Display results
    st.write("### Prediction:")
    if "Fresh" in label:
        st.success(f"The fruit is {label} with {confidence*100:.2f}% confidence.")
    else:
        st.error(f"The fruit is {label} with {confidence*100:.2f}% confidence.")


Overwriting streamlit_app.py


In [15]:
#kill any running streamlit apps

In [16]:
%%writefile streamlit_app.py
import streamlit as st
import numpy as np
import tensorflow as tf
from PIL import Image

st.markdown("""
<style>
/* General Body and Main Container Styling for Dark Theme */
body {
    background-color: #1a1a1a; /* Dark background */
    color: #e0e0e0; /* Light text */
    font-family: 'Roboto', sans-serif; /* Elegant font, fallback to sans-serif */
}

.main .block-container {
    padding-top: 2rem;
    padding-bottom: 2rem;
    padding-left: 1rem;
    padding-right: 1rem;
    max-width: 700px; /* Limit width for centered content */
    margin: auto; /* Center content */
}

/* Header Styling */
h1, h2, h3, h4, h5, h6 {
    color: #4CAF50; /* Eye-catching green for headers */
    font-weight: bold;
    text-align: center; /* Center headers */
    padding-bottom: 0.5rem;
    border-bottom: 2px solid #333; /* Subtle line below headers */
    margin-bottom: 1.5rem;
}

/* Buttons */
.stButton>button {
    background-color: #4CAF50;
    color: white;
    border-radius: 8px;
    border: none;
    padding: 10px 20px;
    font-size: 16px;
    cursor: pointer;
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.3);
    transition: all 0.3s ease-in-out;
    width: 100%; /* Make buttons span full width */
    margin-top: 1rem;
}

.stButton>button:hover {
    background-color: #5cb85c;
    box-shadow: 0 6px 12px rgba(0, 255, 0, 0.4); /* Green glow effect */
    transform: translateY(-2px);
}

/* File Uploader and Camera Input Preview Styling */
.stFileUploader, .stCameraInput {
    border: 2px dashed #4CAF50;
    border-radius: 12px;
    padding: 20px;
    text-align: center;
    background-color: #2a2a2a;
    transition: all 0.3s ease;
    margin-bottom: 1.5rem;
}

.stFileUploader:hover, .stCameraInput:hover {
    background-color: #3a3a3a;
    border-color: #66bb6a;
}

.stImage { /* Style for displayed images */
    border-radius: 12px;
    box-shadow: 0 8px 16px rgba(0, 0, 0, 0.5);
    margin: 1.5rem auto; /* Center image and provide space */
    display: block; /* Ensure margin auto works */
    max-width: 100%;
    height: auto;
}

/* Text and general elements */
p, label, .stMarkdown, .css-1jc7o2r, .css-1l0bqyk, .css-1vq4p4u, .css-1qxtsq5, .css-1d391kg {
    color: #c0c0c0;
    line-height: 1.6;
}

/* Success/Error messages */
.stAlert {
    border-radius: 8px;
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);
}
.stAlert.success {
    background-color: #d4edda;
    color: #155724;
}
.stAlert.error {
    background-color: #f8d7da;
    color: #721c24;
}

/* Streamlit's main content area */
.stApp {
    background-color: #1a1a1a;
}

/* Responsiveness */
@media (max-width: 768px) {
    .main .block-container {
        padding-left: 0.5rem;
        padding-right: 0.5rem;
    }
}
</style>
""", unsafe_allow_html=True)

# Load the pre-trained model
# Ensure the model file is accessible (e.g., in the same directory or specified path)
MODEL_PATH = 'fruit_fresh_spoiled_model.h5'

@st.cache_resource
def load_my_model():
    model = tf.keras.models.load_model(MODEL_PATH)
    return model

model = load_my_model()

IMG_SIZE = (150, 150)

def classify_fruit_streamlit(img):
    """Predicts whether a fruit image is fresh or spoiled."""
    # Ensure image is in PIL format for consistent processing
    if isinstance(img, np.ndarray):
        img = Image.fromarray(img)

    # Resize and convert to RGB
    img = img.convert("RGB").resize(IMG_SIZE)

    # Normalize and expand dimensions
    x = np.expand_dims(np.array(img) / 255.0, axis=0);

    # Make prediction
    prediction = model.predict(x)[0][0]

    # Determine label and confidence
    if prediction < 0.5:
        label = "🍏 Fresh"
        confidence = 1 - prediction
    else:
        label = "🍂 Spoiled"
        confidence = prediction

    return label, confidence

# Streamlit App Layout
st.title("🍎 Fresh vs Spoiled Fruit Detector")
st.write("Upload an image of a fruit or use your camera to classify it as fresh or spoiled.")

uploaded_file = st.file_uploader("Choose an image from your files...", type=["jpg", "jpeg", "png"])
camera_image = st.camera_input("Take a picture with your camera")

image_to_process = None
if uploaded_file is not None:
    image_to_process = Image.open(uploaded_file)
elif camera_image is not None:
    image_to_process = Image.open(camera_image)

if image_to_process is not None:
    # Display the selected image
    st.image(image_to_process, caption='Selected Image', use_container_width=True)
    st.write("")

    # Make prediction
    label, confidence = classify_fruit_streamlit(image_to_process)

    # Display results
    st.write("### Prediction:")
    if "Fresh" in label:
        st.success(f"The fruit is {label} with {confidence*100:.2f}% confidence.")
    else:
        st.error(f"The fruit is {label} with {confidence*100:.2f}% confidence.")

Overwriting streamlit_app.py


In [17]:
import subprocess

# Find and kill any processes named 'streamlit'
try:
    # List all processes and filter for streamlit
    # os.kill is for a specific PID, ps aux | grep is better for finding by name
    # This is a common way to kill previous streamlit processes in a Colab-like environment
    output = subprocess.check_output(['ps', 'aux']).decode('utf-8')
    for line in output.splitlines():
        if 'streamlit run' in line and 'python' in line:
            pid = int(line.split()[1])
            subprocess.run(['kill', '-9', str(pid)])
            print(f"Killed Streamlit process with PID: {pid}")
except Exception as e:
    print(f"No Streamlit processes found to kill or an error occurred: {e}")

# Also kill any ngrok processes
from pyngrok import ngrok
ngrok.kill()
print("✅ ngrok processes killed.")

✅ ngrok processes killed.


In [18]:
%%writefile streamlit_app.py
import streamlit as st
import numpy as np
import tensorflow as tf
from PIL import Image

st.markdown("""
<style>
/* General Body and Main Container Styling for Dark Theme */
body {
    background-color: #1a1a1a; /* Dark background */
    color: #e0e0e0; /* Light text */
    font-family: 'Roboto', sans-serif; /* Elegant font, fallback to sans-serif */
}

.main .block-container {
    padding-top: 2rem;
    padding-bottom: 2rem;
    padding-left: 1rem;
    padding-right: 1rem;
    max-width: 700px; /* Limit width for centered content */
    margin: auto; /* Center content */
}

/* Header Styling */
h1, h2, h3, h4, h5, h6 {
    color: #4CAF50; /* Eye-catching green for headers */
    font-weight: bold;
    text-align: center; /* Center headers */
    padding-bottom: 0.5rem;
    border-bottom: 2px solid #333; /* Subtle line below headers */
    margin-bottom: 1.5rem;
}

/* Buttons */
.stButton>button {
    background-color: #4CAF50;
    color: white;
    border-radius: 8px;
    border: none;
    padding: 10px 20px;
    font-size: 16px;
    cursor: pointer;
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.3);
    transition: all 0.3s ease-in-out;
    width: 100%; /* Make buttons span full width */
    margin-top: 1rem;
}

.stButton>button:hover {
    background-color: #5cb85c;
    box-shadow: 0 6px 12px rgba(0, 255, 0, 0.4); /* Green glow effect */
    transform: translateY(-2px);
}

/* File Uploader and Camera Input Preview Styling */
.stFileUploader, .stCameraInput {
    border: 2px dashed #4CAF50;
    border-radius: 12px;
    padding: 20px;
    text-align: center;
    background-color: #2a2a2a;
    transition: all 0.3s ease;
    margin-bottom: 1.5rem;
}

.stFileUploader:hover, .stCameraInput:hover {
    background-color: #3a3a3a;
    border-color: #66bb6a;
}

.stImage { /* Style for displayed images */
    border-radius: 12px;
    box-shadow: 0 8px 16px rgba(0, 0, 0, 0.5);
    margin: 1.5rem auto; /* Center image and provide space */
    display: block; /* Ensure margin auto works */
    max-width: 100%;
    height: auto;
}

/* Text and general elements */
p, label, .stMarkdown, .css-1jc7o2r, .css-1l0bqyk, .css-1vq4p4u, .css-1qxtsq5, .css-1d391kg {
    color: #c0c0c0;
    line-height: 1.6;
}

/* Success/Error messages */
.stAlert {
    border-radius: 8px;
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);
}
.stAlert.success {
    background-color: #d4edda;
    color: #155724;
}
.stAlert.error {
    background-color: #f8d7da;
    color: #721c24;
}

/* Streamlit's main content area */
.stApp {
    background-color: #1a1a1a;
}

/* Responsiveness */
@media (max-width: 768px) {
    .main .block-container {
        padding-left: 0.5rem;
        padding-right: 0.5rem;
    }
}
</style>
""", unsafe_allow_html=True)

# Load the pre-trained model
MODEL_PATH = 'fruit_fresh_spoiled_model.h5'

@st.cache_resource
def load_my_model():
    model = tf.keras.models.load_model(MODEL_PATH)
    return model

model = load_my_model()

IMG_SIZE = (150, 150)

def classify_fruit_streamlit(img):
    """Predicts whether a fruit image is fresh or spoiled."""
    if isinstance(img, np.ndarray):
        img = Image.fromarray(img)

    img = img.convert("RGB").resize(IMG_SIZE)
    x = np.expand_dims(np.array(img) / 255.0, axis=0)
    prediction = model.predict(x)[0][0]

    if prediction < 0.5:
        label = "🍏 Fresh"
        confidence = 1 - prediction
    else:
        label = "🍂 Spoiled"
        confidence = prediction

    return label, confidence

# Streamlit App Layout
st.title("🍎 Fresh vs Spoiled Fruit Detector")
st.write("Upload an image of a fruit or use your camera to classify it as fresh or spoiled.")

# Input selection radio button
input_source = st.radio(
    "Select input source:",
    ('Upload Picture', 'Camera Input'),
    key="input_selector"
)

uploaded_file = None
camera_image = None

if input_source == 'Upload Picture':
    uploaded_file = st.file_uploader("Choose an image from your files...", type=["jpg", "jpeg", "png"])
elif input_source == 'Camera Input':
    camera_image = st.camera_input("Take a picture with your camera")

image_to_process = None
if uploaded_file is not None:
    image_to_process = Image.open(uploaded_file)
elif camera_image is not None:
    image_to_process = Image.open(camera_image)

if image_to_process is not None:
    st.image(image_to_process, caption='Selected Image', use_container_width=True)
    st.write("")

    label, confidence = classify_fruit_streamlit(image_to_process)

    st.write("### Prediction:")
    if "Fresh" in label:
        st.success(f"The fruit is {label} with {confidence*100:.2f}% confidence.")
    else:
        st.error(f"The fruit is {label} with {confidence*100:.2f}% confidence.")

Overwriting streamlit_app.py


In [19]:
import subprocess

# Find and kill any processes named 'streamlit'
try:
    # List all processes and filter for streamlit
    # os.kill is for a specific PID, ps aux | grep is better for finding by name
    # This is a common way to kill previous streamlit processes in a Colab-like environment
    output = subprocess.check_output(['ps', 'aux']).decode('utf-8')
    for line in output.splitlines():
        if 'streamlit run' in line and 'python' in line:
            pid = int(line.split()[1])
            subprocess.run(['kill', '-9', str(pid)])
            print(f"Killed Streamlit process with PID: {pid}")
except Exception as e:
    print(f"No Streamlit processes found to kill or an error occurred: {e}")

# Also kill any ngrok processes
from pyngrok import ngrok
ngrok.kill()
print("✅ ngrok processes killed.")

✅ ngrok processes killed.


In [20]:
import subprocess
import time
from pyngrok import ngrok

# Start the Streamlit app in the background
# The app will run on port 8501 by default
streamlit_process = subprocess.Popen(["streamlit", "run", "streamlit_app.py"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
print("✅ Streamlit app started in the background.")

# Wait a moment for Streamlit to start
time.sleep(5)

# Establish an ngrok tunnel to the Streamlit app (port 8501)
try:
    public_url = ngrok.connect(8501)
    print(f"✅ ngrok tunnel established: {public_url}")
    print("You can now access your Streamlit app at the URL above.")
    print("To stop the tunnel and app, interrupt this cell (Ctrl+C).")

    # Keep the script running to maintain the tunnel
    while True:
        time.sleep(1)
except Exception as e:
    print(f"❌ Error establishing ngrok tunnel: {e}")
finally:
    # Ensure Streamlit process is terminated when the tunnel is closed or script interrupted
    if streamlit_process.poll() is None:
        streamlit_process.terminate()
        print("✅ Streamlit app terminated.")
    ngrok.kill()
    print("✅ ngrok process killed.")

✅ Streamlit app started in the background.
✅ ngrok tunnel established: NgrokTunnel: "https://garnett-placeless-donna.ngrok-free.dev" -> "http://localhost:8501"
You can now access your Streamlit app at the URL above.
To stop the tunnel and app, interrupt this cell (Ctrl+C).
✅ Streamlit app terminated.
✅ ngrok process killed.


KeyboardInterrupt: 

In [ ]:
!pip freeze > requirements.txt

In [21]:
model.save("fruit_fresh_spoiled_model.keras")
print("✅ Model saved in native Keras format (fruit_fresh_spoiled_model.keras)")

✅ Model saved in native Keras format (fruit_fresh_spoiled_model.keras)


In [22]:
print("Hi")

Hi
